# Important notice: any use of generative AI for completing the assignment is strictly prohibited.

### Note: if working in Colab, don't forget to select runtime type in Colab: GPU

In [ ]:
import os
import numpy as np
from sklearn import preprocessing
from sklearn.mixture import GaussianMixture as GMM
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KernelDensity
from sklearn.metrics import accuracy_score, f1_score

import math as ma

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import statistics
from six import StringIO

from itertools import chain
import shutil

In [ ]:
# use that if working in colab
# permit Colab access your google drive, and select Princeton account

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# you should be added as viewer to shared Google drive "ECE477 datasets"
#  at https://drive.google.com/drive/u/0/folders/0ABIZHKB-QPnRUk9PVA

!unzip "/content/drive/Shared drives/ECE477 datasets/Assignment4/three-way_classification_real_data.zip" -d "."

## Warning: to ensure the reproducibility of your results and to achieve the full grade, do not change or remove RANDOM_STATE variables and setting random seed statements. If you remove or change them, you may not get the full grade. 

# Part 1: synthetic data generation

## Define generators.

We will use Kernel Density Estimation, Gaussian Mixture Models and Multivariate Normal Distribution methods to generate synthetic data from the read data distribtion.

In [ ]:
import random

RANDOM_STATE = 0
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Task: fit Gaussian Mixture Model (GMM) (2 points)

To get full points, you also should report correct best number of components later (when prompted)

In [ ]:
# Generating synthetic data based on GMM model - The number of Gaussians is selected based on maximizing
# the sum of the log-probabilities of instances in the validation set
def GMM_sample_generation(X_train, X_validation):
    max_components = 5
    n_components = np.arange(1, max_components, 1)
    sum_array = np.zeros((max_components - 1))
    for i in n_components:
        # define and fit GMM model with covariance_type full, RANDOM_STATE
        # your code is here
        model = ...
        ...
        
        # calc Log-likelihood over validation dataset for i components 
        # and save it to sum_array.
        # hint: call .score_samples()
        # your code is here
        log_likelihoods = ...
        sum_array[i - 1] = ...
        print(f"Accumulated log likelihood for {i} components:", sum_array[i - 1])
        
    n_components = np.argmax(sum_array)
    n_components = n_components + 1
    print("Number of components: {}".format(n_components))

    # define and fit the model with best number of components
    # your code is here
    gmm = ...
    ...
    print(gmm.converged_)

    out = gmm.score(X_validation)
    print(out)
 
    X_syn = gmm.sample(100000)
    print(X_syn[0].shape)
    return X_syn

### Task 1a: fit Kernel Density Estimation Model (KDE) (3 points)

To get full points, you also should report correct best bandwidth later (when prompted)

In [ ]:
# Generating synthetic data based on KDE model - The bandwidth is selected based on maximizing
# the sum of the log-probabilities of instances in the validation set
def KDE_sample_generation(X_train, X_validation):
    bw_list = [0.01, 0.05, 0.1, 0.2, 0.3]
    log_like = np.zeros((len(bw_list)))
    
    # fit KDE with gaussian kernel and all bandwidth, score them on validation dataset
    # and store their score at log_like array
    for bw in bw_list:
        # your code is here
        
    j = np.argmax(log_like)
    # fit the model with the highest score 
    # your code is here
    kde = ...
    ...
    out = kde.score(X_validation)
    print(out)
    X_syn = kde.sample(100000)
    print(X_syn.shape)

    return X_syn

In [ ]:
def sample_generation (X_train, X_validation, method):
    """supported methods: GMM, KDE"""
    count = 100000
    if method == "GMM":
        X_syn_out = GMM_sample_generation(X_train, X_validation)
        X_syn = X_syn_out[0]
    elif method == "KDE":
        X_syn = KDE_sample_generation(X_train, X_validation)
    else:
        print ("method not supported")
    return X_syn

In [ ]:
path_to_real_data = 'three-way_classification_real_data'

In [ ]:
# Loading the data
X_train, y_train, X_validation, y_validation = (
        np.load(os.path.join(path_to_real_data, 'X_train.npy')),
        np.load(os.path.join(path_to_real_data, 'y_train.npy')),
        np.load(os.path.join(path_to_real_data, 'X_validation.npy')),
        np.load(os.path.join(path_to_real_data, 'y_validation.npy')),
)

X_train_original = X_train
X_validation_original = X_validation

X_train_original.shape, X_validation_original.shape

In [ ]:
# Defining the feature dictionary for various sensors for real data
featdict = {
    "GSR": list(np.arange(3900,3960)),
    "Temp": list(np.arange(3960,4020)),
    "IBI" : list(np.arange(4080,4140)),
    "Ox" : list(np.arange(4146,4147)),
    "BP" : list(np.arange(4160,4162)),
    "Q": list(np.arange(4147,4158))
}

## Generating the synthetic data based on 6 set of features

In [ ]:
indexes = np.array(featdict["GSR"] + featdict["Temp"] + featdict["IBI"] + featdict["Ox"] + featdict ["BP"] + featdict["Q"]).reshape(-1)
X_train  = X_train_original[:, indexes]
X_validation = X_validation_original[:, indexes]

X_train.shape, X_validation.shape

### Task 1b: fitting Random Forest Classifier (2 points)

#### We fit RF on real data, and then label synthetically generated data with this classifier

To get full points, you also should report correct best tree depth later (when prompted)

In [ ]:
# Finding the most accurate model on the validation set, will be used later to label the synthetic data
validation_acc = []
train_acc = []

RANDOM_STATE_RF = 11

print("Fitting Random Forest Classifier")
for i in range (1, 10):
    print(f'current max_depth: {i}')
    # fit Random Forest with Gini criterion, current max_depth and RANDOM_STATE_RF as random_state
    # store its validation accuracy (call "accuracy_score" function to calculate it)
    # your code is here
    
print (validation_acc)
tree_depth = np.argmax(validation_acc) + 1
print("Best tree depth", tree_depth)

# fit the best performing classifier here, we will use it further
# make sure to set RANDOM_STATE_RF
# your code is here
clf = ...

### Report the best tree depth:

you answer is here

In [ ]:
# Selecting the method of synthetic data generation:
method = 'GMM'
X_syn = sample_generation(X_train, X_validation, method)
#Labeling the synthetic data basesd on the most accurate random forest model on the validation set
y_syn = np.zeros((X_syn.shape[0]))
y_syn = clf.predict(X_syn)
print(y_syn.shape)

### Report the best number of components:

you answer is here

In [ ]:
# Selecting the method of synthetic data generation:
method = 'KDE'
X_syn = sample_generation(X_train, X_validation, method)

#Labeling the synthetic data based on the most accurate random forest model on the validation set
y_syn = np.zeros((X_syn.shape[0]))
y_syn = clf.predict(X_syn)
print (y_syn.shape)

### Report the best bandwidth:

you answer is here

In [ ]:
path_synthetic_data = 'synthetic_data_generated'

In [ ]:
# save synthetic datasets
for method in ('GMM', 'KDE'):
    path_to_save = os.path.join(path_synthetic_data, method)
    os.makedirs(path_to_save, exist_ok=True)
    np.save(os.path.join(path_to_save, 'X-syn.npy'), X_syn)
    np.save(os.path.join(path_to_save, 'y-syn.npy'), y_syn)

# Part 2. Training

In [ ]:
# definig the device variable to use GPU if it is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Task 2a: define two missing network layers and metrcis (2 points)

In [ ]:
# Network and function definitions

class DynamicNet5(torch.nn.Module):
    def __init__(self, D_in, H_1=256, H_2=128, H_3=128, D_out=4):
        super(DynamicNet5, self).__init__()
        # your code is here
        self.fc2 = ...
        ...

        self.fc4 = nn.Linear(H_3, D_out)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = F.leaky_relu(self.fc3(x))
        y_pred = (self.fc4(x))
        return y_pred
    
    
def metrics (TP, TN, FP, FN):
    # your code is here

    return F1, FPR, FNR

### Reload real data for training

In [ ]:
X_train, y_train, X_validation, y_validation, X_test, y_test = (
        np.load(os.path.join(path_to_real_data, 'X_train.npy')),
        np.load(os.path.join(path_to_real_data, 'y_train.npy')),
        np.load(os.path.join(path_to_real_data, 'X_validation.npy')),
        np.load(os.path.join(path_to_real_data, 'y_validation.npy')),
        np.load(os.path.join(path_to_real_data, 'X_test.npy')),
        np.load(os.path.join(path_to_real_data, 'y_test.npy')),
)

# Copying the data into new variables
# this is done since we later want to different feature sets of our data
X_train_original = X_train
X_validation_original = X_validation
X_test_original = X_test

### Load synthetic data
In this asssignment, we will use only KDE generated data (you should have generated them on the previous step).

In [ ]:
method = 'KDE'

X_syn = np.load (f'{path_synthetic_data}/{method}/X-syn.npy')
y_syn = np.load (f'{path_synthetic_data}/{method}/y-syn.npy')

X_syn_original = X_syn

#feature dictinary for synthetic data|
featdict_syn = {
    "GSR": list(np.arange(0,60)),
    "Temp": list(np.arange(60,120)),
    "IBI" : list(np.arange(120,180)),
    "Ox" : list(np.arange(180,181)),
    "BP" : list(np.arange(181,183)),
    "Q": list(np.arange(183,194))
}

### Generate all the subsets of feature categories
Since covidDeep has 6 feature categories, we generate all the subsets of these 6 categories here

In [ ]:
# get feature combinations for training

combs_array = [
    ('GSR', 'Temp', 'IBI', 'Ox', 'BP'),
    ('Temp', 'IBI', 'Ox', 'BP', 'Q'),
    ('GSR', 'Temp', 'Ox', 'BP', 'Q'),
    ('GSR', 'Temp', 'IBI', 'Ox', 'BP', 'Q'),
]

## Train DNN-1 models (models trained on real data)

In [ ]:
path_to_save_models = 'trained_models'

# DNN-1 models are ones trained on real data
path_dnn1 = os.path.join(path_to_save_models, 'DNN-1')
os.makedirs(path_dnn1, exist_ok=True)

### Task 2b: define optimizer and classification loss (2 points)

optimizer: Adam, learning rate: 1e-3
classification loss: Cross Entropy

In [ ]:

# ==============
# Random states:
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

torch.manual_seed(RANDOM_STATE)
torch.cuda.manual_seed(RANDOM_STATE)
# ==============

saved_metrics = {
    key: {} for key in combs_array
}


for cmb in combs_array:
    indices = list()

    print("For features:", cmb)
    for feat in cmb:
        indices += featdict[feat]
    # corresponding indices in the real data
    indices = np.array(indices).reshape(-1)

    # considering a subset of features y slicing the feature space
    X_train  = X_train_original[:, indices]
    X_validation = X_validation_original[:, indices]
    X_test = X_test_original[:, indices]

    # converting the data into torch tensors
    x = torch.from_numpy(X_train).float()
    y = torch.from_numpy(y_train.reshape(-1)).long()

    # putting the data on 'device'
    x = x.to(device)
    y = y.to(device)

    m = X_train.shape[1]
    print ("Number of features: ", m)
    model = DynamicNet5(m)
    model = model.to(device)

    
    # set ADAM optimizer with lr= 1e-3 and loss
    # your code is here
    criterion = ...
    optimizer = ...

    # training loop (the number of epochs can be explored)
    print("Training loop...")
    for t in range(1000):
        optimizer.zero_grad()
        loss = criterion(model(x), y)
        loss.backward()
        optimizer.step()

    # define confusion matrix
    cm = np.zeros((4, 4), dtype=int)
    TP = TN = FP = FN = 0

    # evaluation on test set without computing gradients (with torch.no_grad())
    y_pred_array = []
    correct = 0
    total = X_test.shape[0]
    print('Running evaluation...')
    with torch.no_grad():
        for i in range(total):
            # Convert single test sample and label to tensors, move to device
            x_t = torch.from_numpy(X_test[i, :]).float().to(device)
            y_t = torch.tensor(y_test[i], dtype=torch.long).to(device)

            # Forward pass through the model
            output = model(x_t)
            # Compute predicted class
            y_pred = torch.argmax(output.cpu(), dim=-1).item()
            y_pred_array.append(y_pred)
            label = int(y_test[i].item())

            # Update the confusion matrix
            cm[label, y_pred] += 1

            # Update correct predictions and binary classification counters
            if y_pred == label:
                correct += 1
                if y_pred == 0:
                    TN += 1
                else:
                    TP += 1
            else:
                if y_pred == 0:
                    FN += 1
                else:
                    FP += 1

    # After processing all samples, extract confusion matrix values:
    T1, T2, T3, T4 = cm[0, 0], cm[1, 1], cm[2, 2], cm[3, 3]
    F12, F13, F14 = cm[0, 1], cm[0, 2], cm[0, 3]
    F21, F23, F24 = cm[1, 0], cm[1, 2], cm[1, 3]
    F31, F32, F34 = cm[2, 0], cm[2, 1], cm[2, 3]
    F41, F42, F43 = cm[3, 0], cm[3, 1], cm[3, 2]

    test_accuracy_original = 100 * correct/total
    print ('Test Accuracy:', round(test_accuracy_original, 3))

    print(f"T1: {T1}, T2: {T2}, T3: {T3}, T4: {T4}")
    print(f"F12: {F12}, F13: {F13}, F14: {F14}")
    print(f"F21: {F21}, F23: {F23}, F24: {F24}")
    print(f"F31: {F31}, F32: {F32}, F34: {F34}")
    print(f"F41: {F41}, F42: {F42}, F43: {F43}")

    print ("True positive: ", TP)
    print ("False positive: ", FP)
    print ("True negative: ", TN)
    print ("False negative: ", FN)

    # computing the performance metrics
    F1, FPR, FNR = metrics(TP, TN, FP, FN)
    print("Aggregated metrics:")
    print ("False positve rate: ",  round(FPR, 3))
    print ("False negative rate: ",  round(FNR, 3))
    print ("F1 Score: ",  round(F1, 3))
    y_pred_array = np.array(y_pred_array)

    F1_original = 100 * f1_score(y_test, y_pred_array, average='macro')
    FPR_original = 100 * ((F12+F13+F14)/(F12+F13+F14+T1)) if (F12+F13+F14+T1) != 0 else np.nan
    FNR2_original = 100 * ((F21+F23+F24)/(F21+F23+F24+T2)) if (F21+F23+F24+T2) != 0 else np.nan
    FNR3_original = 100 * ((F31+F32+F34)/(F31+F32+F34+T3)) if (F31+F32+F34+T3) != 0 else np.nan
    FNR4_original = 100 * ((F41+F42+F43)/(F41+F42+F43+T4)) if (F41+F42+F43+T4) != 0 else np.nan

    print("Per class metrics:")
    print ("False positve rate: 1",  round(FPR_original, 3))
    print ("False negative rate 2: ",  round(FNR2_original, 3))
    print ("False negative rate 3",  round(FNR3_original, 3))
    print ("False negative rate 4: ",  round(FNR4_original, 3))
    print ("F1 Score: ",  round(F1_original, 3))

    saved_metrics[cmb] = {
        'acc': round(test_accuracy_original, 3),
        'FPR': round(FPR, 3),
        'FNR': round(FNR, 3),
        'F1': round(F1_original, 3),
    }

    # Saving the nodel for DNN 1, using the name of features used in naming the saved file
    save_to = os.path.join(path_dnn1, '_'.join(cmb) + '.pth')
    torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, save_to)

    print(f'saved to {save_to}\n')

### Task 2c: class 4 metrics (1 point)
Why do you think model never misclassifies class 4 as other classes (neither FP, neither FN)?

**your answer is here**

## Train DNN-2 models (models pretrained on synthetic data and trained on real data)

In this part, the model is pretrained on synthetic data generated in the first part for small number of epochs, and then trained on real data.

In [ ]:
# DNN-2 models are ones pretrained with synthetic and then trained on real data
path_dnn2 = os.path.join(path_to_save_models, 'DNN-2')
os.makedirs(path_dnn2, exist_ok=True)

### Task 2d: define training loops (3 points)

(1) Define optimizer and loss as in the previous part; 

(2) Define pretraining on synthetic data and then (3) training or real data (fill in 3 gaps)

In [ ]:
# ==============
# Random states:
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

torch.manual_seed(RANDOM_STATE)
torch.cuda.manual_seed(RANDOM_STATE)
# ==============

saved_metrics2 = {
    key: {} for key in combs_array
}

for cmb in combs_array:
    indices = list()
    indices_syn = list()
    
    print("For features:", cmb)
    for feat in cmb:
        indices += featdict[feat]
        indices_syn += featdict_syn[feat]
    # corresponding indices in the real data
    indices = np.array(indices).reshape(-1)
    # corresponding indices in the synthetic data
    indices_syn = np.array(indices_syn).reshape(-1)

    # considering a subset of features y slicing the feature space
    X_train  = X_train_original[:, indices]
    X_validation = X_validation_original[:, indices]
    X_test = X_test_original[:, indices]

    X_syn = X_syn_original  [:, indices_syn]

    print("Unique lables in train:", np.unique(y_train))
    print("Unique lables in validation:", np.unique(y_validation))

    # converting the data into torch tensors
    x = torch.from_numpy(X_syn).float()
    y = torch.from_numpy(y_syn.reshape(-1)).long()

    # putting the data on 'device'
    x = x.to(device)
    y = y.to(device)
    
    m = X_train.shape[1]
    print ("Number of features: ", m)
    model = DynamicNet5(m)
    model = model.to(device)

    # (1) again, set ADAM optimizer with lr= 1e-3  and CrossEntropy loss
    # your code is here

    # (2) After setting optimizer and loss, define pretraining with synthetic data for 50 epochs. 
    # Hint: check the train loop for DNN1
    # (we do pretraining usually for fewer epochs compared to final training with real data)
    # your code is here

    m, n = X_train.shape
    m2, n2 = X_validation.shape
    X_all = np.zeros((m + m2, n))
    y_all = np.zeros((m + m2))

    X_all[0:m, :] = X_train
    X_all[m: m + m2, :] = X_validation
    y_all[0:m] = y_train.reshape(-1)
    y_all[m:m + m2] = y_validation.reshape(-1)

    x = torch.from_numpy(X_all).float()
    y = torch.from_numpy(y_all.reshape(-1)).long()
    x = x.to(device)
    y = y.to(device)
    
    
    # (3) define training with real data for 1000 epochs. Hint: check the train loop for DNN1
    # your code is here
        
    # define confusion matrix
    cm = np.zeros((4, 4), dtype=int)
    TP = TN = FP = FN = 0
    
    # evaluation on test data
    y_pred_syn_array = list()
    correct = 0
    total = X_test.shape[0]
    print('Running evaluation...')
    with torch.no_grad():
        for i in range(X_test.shape[0]):
            x_t = torch.from_numpy(X_test[i, :]).float().to(device)
            y_t = torch.tensor(y_test[i], dtype=torch.long, device=device)
            
            output = model.forward(x_t)
            np_output = (output.cpu()).numpy()
            y_pred = np.argmax(np_output)
            y_pred_syn_array.append(y_pred)
            label = int(y_test[i].item())
            
            # Update the confusion matrix
            cm[label, y_pred] += 1

            # Update correct predictions and binary classification counters
            if y_pred == label:
                correct += 1
                if y_pred == 0:
                    TN += 1
                else:
                    TP += 1
            else:
                if y_pred == 0:
                    FN += 1
                else:
                    FP += 1
                    
        # After processing all samples, extract confusion matrix values:
        T1, T2, T3, T4 = cm[0, 0], cm[1, 1], cm[2, 2], cm[3, 3]
        F12, F13, F14 = cm[0, 1], cm[0, 2], cm[0, 3]
        F21, F23, F24 = cm[1, 0], cm[1, 2], cm[1, 3]
        F31, F32, F34 = cm[2, 0], cm[2, 1], cm[2, 3]
        F41, F42, F43 = cm[3, 0], cm[3, 1], cm[3, 2]
        
        test_accuracy_syn = 100 * correct / total
        print ('Test Accuracy:', round(test_accuracy_syn, 3))

        print(f"T1: {T1}, T2: {T2}, T3: {T3}, T4: {T4}")
        print(f"F12: {F12}, F13: {F13}, F14: {F14}")
        print(f"F21: {F21}, F23: {F23}, F24: {F24}")
        print(f"F31: {F31}, F32: {F32}, F34: {F34}")
        print(f"F41: {F41}, F42: {F42}, F43: {F43}")

        print("Aggregated metrics:")
        print ("True positive: ", TP)
        print ("False positive: ", FP)
        print ("True negative: ", TN)
        print ("False negative: ", FN)
        
        F1, FPR, FNR = metrics(TP, TN, FP, FN)
        print("False positve rate: ", round(FPR, 3))
        print("False negative rate: ", round(FNR, 3))
        print("F1 Score: ", round(F1, 3))
        y_pred_syn_array = np.array(y_pred_syn_array)

        # computing the performance metrics
        F1_syn = 100 * f1_score(y_test, y_pred_syn_array, average='macro') # .item()
        FPR_syn = 100 * ((F12 + F13 + F14) / (F12 + F13 + F14 + T1)) if (F12 + F13 + F14 + T1) else np.nan
        FNR2_syn = 100 * ((F21 + F23 + F24) / (F21 + F23 + F24 + T2)) if (F21 + F23 + F24 + T2) else np.nan
        FNR3_syn = 100 * ((F31 + F32 + F34) / (F31 + F32 + F34 + T3)) if (F31 + F32 + F34 + T3) else np.nan
        FNR4_syn = 100 * ((F41 + F42 + F43) / (F41 + F42 + F43 + T4)) if (F41 + F42 + F43 + T4) else np.nan

        print("Per-class metrics:")
        print ("False positve rate: 1",  round(FPR_syn, 3))
        print ("False negative rate 2: ",  round(FNR2_syn, 3))
        print ("False negative rate 3",  round(FNR3_syn, 3))
        print ("False negative rate 4: ",  round(FNR4_syn, 3))
        print ("F1 Score: ",  round(F1_syn, 3))

        saved_metrics2[cmb] = {
            'acc': round(test_accuracy_syn, 3),
            'FPR': round(FPR, 3),
            'FNR': round(FNR, 3),
            'F1': round(F1_syn, 3),
        }

        #saving the DNN 2 models, with the name of the features used in the name of the file        
        save_to = os.path.join(path_dnn2, '_'.join(cmb) + '.pth')
        torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
                }, save_to)

        print(f'saved to {save_to}\n')

### Task 2e: find the best feature (2 points)
Write code that finds the feature combinatinons (like ('GSR', 'IBI', 'Ox', 'BP')) with:

1) best accuracy
2) best F1 score

Note that your results may differ from the CovidDeep paper, since we changed hyperparameters to make training faster for educational purposes.

In [ ]:
hint: look where we saved metrics in the code above

def find_best_features(saved_metrics: dict) -> None:
    # your code is here
    
    print("Features with max accuracy:", max_acc)
    print("Maximum accuracy:", max_acc)
    
    # your code is here
    print("Features with max F1 score:", max_acc)
    print("Maximum F1 score:", max_acc)

### Task 2f: Report accuracies for both DNN1 and DNN2 (1 point)

Using find_best_features, report best F1 and accuracy scores, and feature combination that performs the best for both DNN1 and DNN2


In [ ]:
# your answer is here

# Part 3. DNN3: Grow and Prune approach.

The starting point can be pretrained DNN 1 and 2 models to apply grow and prune synthetic to improve the classification performance. However, we can also leverage grow and prune synthesis from scratch

In [ ]:
def load_data_train(data_path='./data/sonar/', mode='all', fold=None):
    if fold != None:
        X_train = np.load(data_path + "X_train" + str(fold) + '.npy')
        y_train = np.load(data_path + "y_train" + str(fold) + '.npy')
        X_test = np.load(data_path + "X_validation"+ str(fold) + '.npy')
        y_test = np.load(data_path + "y_validation"+ str(fold) + '.npy')
         
    else:
        if mode == 'all':
            X_train = np.load(data_path + "X_train.npy")
            y_train = np.load(data_path + "y_train.npy")
        elif mode == 'easy':
            X_train = np.load(data_path + "X_easy.npy")
            y_train = np.load(data_path + "y_easy.npy")
        elif mode == 'hard':
            X_train = np.load(data_path + "X_hard.npy")
            y_train = np.load(data_path + "y_hard.npy")
        
        X_test = np.load(data_path + "X_validation.npy")
        y_test = np.load(data_path + "y_validation.npy")
    

    X_train = X_train.astype(float)
    X_test = X_test.astype(float)
    y_train = y_train.astype(int)
    y_test = y_test.astype(int)
    
    return X_train, y_train, X_test, y_test


def load_data_test(data_path='./data/sonar/', fold=None):
    if fold != None:
        X_train = np.load(data_path + "X_train" + str(fold) + '.npy')
        y_train = np.load(data_path + "y_train" + str(fold) + '.npy')
        X_test = np.load(data_path + "X_test"+ str(fold) + '.npy')
        y_test = np.load(data_path + "y_test"+ str(fold) + '.npy')
        
         
    else:
        X_train = np.load(data_path + "X_train_total.npy")
        X_test = np.load(data_path + "X_test.npy")
        y_train = np.load(data_path + "y_train_total.npy")
        y_test = np.load(data_path + "y_test.npy")

    X_train = X_train.astype(float)
    X_test = X_test.astype(float)
    y_train = y_train.astype(int)
    y_test = y_test.astype(int)
    

    return X_train, y_train, X_test, y_test

In [ ]:
def loadData(path_to_dataset, indexes, mode='train', batch_size=64, generator=None):
    if mode == 'train':
        X_train, y_train, X_validation, y_validation = load_data_train(f"{path_to_dataset}/")
    elif mode == 'test':
        X_train, y_train, X_validation, y_validation = load_data_test(f"{path_to_dataset}/")
    
    print('number of features:', len(indexes))
    
    #Only using these features
    X_train  = X_train[:, indexes]
    X_validation = X_validation[:, indexes]
    
    print('number of labels in train:', np.unique(y_train))
    print('number of labels in validation:', np.unique(y_validation))

    # changing to torch tensors
    X_train_tensor = torch.from_numpy(X_train).float()
    y_train_tensor = torch.from_numpy(y_train.reshape(-1)).long()
    X_validation_tensor = torch.from_numpy(X_validation).float()
    y_validation_tensor = torch.from_numpy(y_validation.reshape(-1)).long()

    # Create TensorDatasets
    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    validation_data = TensorDataset(X_validation_tensor, y_validation_tensor)

    # Create DataLoaders
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, generator=generator)
    validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False, generator=generator)

    return train_loader, validation_loader

In [ ]:
path_dnn3 = os.path.join(path_to_save_models, 'DNN-3')
os.makedirs(path_dnn3, exist_ok=True)


def save_checkpoint(state, path_to_save, is_best, name, filename='_checkpoint.pth.tar'):
    torch.save(state, os.path.join(path_to_save, name + filename))
    print(f"saved to {os.path.join(path_to_save, name + filename)}")
    if is_best:
        shutil.copyfile(os.path.join(path_to_save, name + filename), os.path.join(path_dnn3, name + '_model_best.pth.tar'))
        print(f"also saved as the best checkpoint to {os.path.join(path_dnn3, name + '_model_best.pth.tar')}")

In [ ]:
def disp_acc():
    global model, trainloader, validationloader
    total = 0
    correct = 0
    
    # For the CovidDeep Confusion matrix, and FPR, FNR, and F1 calculations
    #Ti: the cohort i instances that are correcly classified
    #Fij: The incorrect predictions, label: i, prediction: j
    cm = np.zeros((3, 3), dtype=int)

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model.myforward(inputs, cw1, cw2, cw3, cw4)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels.data).sum()
        total += labels.size(0)
    print('Train: %d/%d' %(correct, total))
    train_acc = correct * 1. / total
    
    total = 0
    correct = 0
    
    y_test = np.zeros((1))
    y_pred_array = np.zeros((1))
    
    for i, data in enumerate(validationloader, 0):
        inputs, targets = data
        inputs, labels = inputs.to(device), targets.to(device)
        outputs = model.myforward(inputs, cw1, cw2, cw3, cw4)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        # Convert GPU tensors to CPU and detach before converting to NumPy
        labels_np = labels.cpu().detach().numpy()
        predicted_np = predicted.cpu().detach().numpy()

        y_test = np.concatenate((y_test, labels_np), axis=0)
        y_pred_array = np.concatenate((y_pred_array, predicted_np), axis=0)

        # Computing the confusion matrix
        for j in range(labels_np.shape[0]):
            cm[labels_np[j], predicted_np[j]] += 1

    T1, T2, T3 = cm[0, 0], cm[1, 1], cm[2, 2]
    F12, F13 = cm[0, 1], cm[0, 2]
    F21, F23 = cm[1, 0], cm[1, 2]
    F31, F32 = cm[2, 0], cm[2, 1]

    
    y_test = np.array(y_test)
    y_pred_array = np.array(y_pred_array)
    y_test = np.delete(y_test, 0, 0)
    y_pred_array = np.delete(y_pred_array, 0, 0)
    
    # Computing the F1 score and FNR and FPR values
    F1_original = 100 * f1_score(y_test, y_pred_array, average='macro')
    FPR_original = 100 * ((F12+F13)/(F12+F13+T1))
    FNR2_original = 100 * ((F21+F23)/(F21+F23+T2))
    FNR3_original = 100 * ((F31+F32)/(F31+F32+T3))
    test_accuracy_original = 100 * correct/total

    print("T1: ", T1)
    print("T2: ", T2)
    print("T3: ", T3)

    print("F12: ", F12)
    print("F13: ", F13)

    print("F21: ", F21)
    print("F23: ", F23)

    print("F31: ", F31)
    print("F32: ", F32)

    print("False positive rate: ", round(FPR_original, 3))
    print("False negative rate 2: ", round(FNR2_original, 3))
    print("False negative rate 3: ", round(FNR3_original, 3))
    print("F1 Score: ", round(F1_original, 3))


    print('Train: %d/%d' %(correct, total))
    validation_acc = correct * 1. /total
    print('Training accuracy: %f, Validation accuracy: %f' 
              % (train_acc, validation_acc))
    return validation_acc

In [ ]:
# loading the model alongside the learned masks
# if the number of layers change, the number of masks need to change and this function can be changed accordingly
def load_model(path_to_load: str, mode='best'):
    global model, name, best_acc_test, global_epoch, cw1, cw2, cw3, cw4   
    if mode == 'best':
        checkpoint = torch.load(path_to_load + '_model_best.pth.tar', weights_only=True)
    elif mode == 'checkpoint':
        checkpoint = torch.load(path_to_load + '_checkpoint.pth.tar', weights_only=True)
    else:
        checkpoint = torch.load(path_to_load + '_prune_checkpoint.pth.tar', weights_only=True)
        
    best_acc_test = checkpoint['best_acc']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    global_epoch = checkpoint['epoch']
    cw1 = checkpoint['cw1']
    cw2 = checkpoint['cw2']
    cw3 = checkpoint['cw3']
    cw4 = checkpoint['cw4']

In [ ]:
def train(duration=10):    
    global best_acc_train, best_acc_test, name, trainloader, validationloader, global_epoch, flagc, best_acc_prune, validation_acc_list
    for epoch in range(global_epoch, global_epoch+duration):  # loop over the dataset multiple times
        for i, data in enumerate(trainloader, 0):
            # get the inputs
            inputs, targets = data
            # move tensors to the configured device
            inputs, labels = inputs.to(device), targets.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = model.myforward(inputs, cw1, cw2, cw3, cw4)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
        total = 0
        correct = 0
        for i, data in enumerate(trainloader, 0):
            inputs, targets = data
            inputs, labels = inputs.to(device), targets.to(device)
            outputs = model.myforward(inputs, cw1, cw2, cw3, cw4)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels.data).sum()
            total += labels.size(0)
        train_acc = correct * 1. / total
        
        total = 0
        correct = 0
        for i, data in enumerate(validationloader, 0):
            inputs, targets = data
            inputs, labels = inputs.to(device), targets.to(device)
            outputs = model.myforward(inputs, cw1, cw2, cw3, cw4)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels.data).sum()
            total += labels.size(0)
        validation_acc = correct * 1. /total
        print('Epoch: %d, Training accuracy: %f, Validation accuracy: %f' 
                  % (epoch, train_acc, validation_acc))
        
        # saving the model with the best validation performance
        if validation_acc > best_acc_test:
            best_acc_test = validation_acc
            best_acc_train = train_acc
            save_checkpoint({
                'epoch': epoch,
                'state_dict': model.state_dict(),
                'best_acc': best_acc_test,
                'optimizer': optimizer.state_dict(),
                'cw1': cw1,
                'cw2': cw2,
                'cw3': cw3,
                'cw4': cw4,
            }, path_dnn3, True, name=name)
        else:
            save_checkpoint({
                'epoch': epoch,
                'state_dict': model.state_dict(),
                'best_acc': best_acc_test,
                'optimizer': optimizer.state_dict(),
                'cw1': cw1,
                'cw2': cw2,
                'cw3': cw3,
                'cw4': cw4,
            }, path_dnn3, False, name=name)

        if (flag == 1) and (validation_acc > best_acc_prune):
            best_acc_prune = validation_acc
            state = {
                     'epoch': epoch,
                     'best_acc': best_acc_prune,
                     'state_dict': model.state_dict(),
                     'optimizer': optimizer.state_dict(),
                     'cw1': cw1,
                     'cw2': cw2,
                     'cw3': cw3,
                     'cw4': cw4,
                    }
            save_checkpoint(state, path_dnn3, False, name = name + '_prune')
            
    global_epoch += duration

In [ ]:
'''
Function: prune connections
Arguments:
    weight_list: weight matrix to prune
    mask_list: corresponding mask to prune
    mode: 'small' prune small weights, 'large' prune large weights, 'rand' random, we normally use it to prune small weights
    num: number of connections left
    percetile: percentile of connections left
'''

def pruneMask(weight_list, mask_list, mode='small', num=100, percentile=None):
    for i, weight in enumerate(weight_list):
        weight_cpu = weight.cpu()
        if i == 0:
            weight_arr = weight_cpu.view(-1).numpy()
        else:
            weight_arr = np.concatenate((weight_arr, weight_cpu.view(-1).numpy()))

    if mode == 'small':
        if percentile:
            threshold = np.percentile(np.absolute(weight_arr), percentile)
        else:
            threshold = np.sort(np.absolute(weight_arr))[-num]
        for i, mask in enumerate(mask_list):
            # Use CPU absolute for threshold comparison; update mask in-place on original device
            mask[torch.abs(weight_list[i]) < float(threshold)] = 0

    elif mode == 'large':
        if percentile:
            threshold = np.percentile(np.absolute(weight_arr), percentile)
        else:
            threshold = np.sort(np.absolute(weight_arr))[-num]
        for i, mask in enumerate(mask_list):
            mask[torch.abs(weight_list[i]) > float(threshold)] = 0
                

'''
Function: add connections
Arguments:
    mask_list: corresponding mask to prune
    mode: 'partial' add part of connections, 'full' restore all connections
    ratio: ratio of active connections after adding connections 
'''
        
def growMask(mask_list, mode='partial', ratio=[0.1,], generator=None):
    if mode == 'partial':
        for i, mask in enumerate(mask_list):
            rand_mat = torch.rand(mask.size(), device=mask.device, generator=generator)
            mask[rand_mat > ratio[i]] = 1
    elif mode == 'full':
        for i, mask in enumerate(mask_list):
            mask.fill_(1)
            
            
def displayConnection(mask_list):
    nonzero_connections = []
    for name, mask in mask_list:
        nonzero_count = np.count_nonzero(mask.cpu().numpy()) if mask.device.type != 'cpu' else np.count_nonzero(mask.numpy())
        nonzero_connections.append(nonzero_count)
        print(f"{name}: {nonzero_count}", end="  ")
    print('\n')
    return nonzero_connections

In [ ]:
# Three hidden layers
# This can be changed based on the loaded architecture 
class Net3(nn.Module):
    def __init__(self, in_num, out_num, hidden_num1, hidden_num2, hidden_num3):
        super(Net3, self).__init__()
        self.fc1 = nn.Linear(in_num, hidden_num1)
        self.fc2 = nn.Linear(hidden_num1, hidden_num2)
        self.fc3 = nn.Linear(hidden_num2, hidden_num3)
        self.fc4 = nn.Linear(hidden_num3, out_num)
    
    # Simple forward pass in a DNN
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
    
    # forward pass that considers binary masks (cw1 to cw4)
    # The number of these masks change as the number of layers change
    def myforward(self, x, cw1, cw2, cw3, cw4):
        self.fc1.weight.data = self.fc1.weight.data * cw1
        self.fc2.weight.data = self.fc2.weight.data * cw2
        self.fc3.weight.data = self.fc3.weight.data * cw3
        self.fc4.weight.data = self.fc4.weight.data * cw4
       
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

## Load the DNN 2 models
The SCANN grow-and-prune is applied to further improve DNN models 2

#### We will work with GSR_Temp_IBI_Ox_BP_Q DNN2 model.

In [ ]:
dnn2_for_grow_prune = 'GSR_Temp_IBI_Ox_BP_Q.pth'
checkpoint = torch.load(os.path.join(path_dnn2, dnn2_for_grow_prune), map_location=torch.device(device), weights_only=True)

### Task 3a: set Net3 model configuration based on the loaded model parameters (2 points)


set in_num, hidden_num1, hidden_num2, hidden_num3, out_num

in_num: shows the number of input features

hidden_num1, hidden_num2, hidden_num3: neurons in hidden layers

out_num: the number of classes

In [ ]:
# # your code is here


# print(in_num, hidden_num1, hidden_num2, out_num)

In [ ]:
# initializing the masks
cw1 = torch.ones([hidden_num1, in_num])
cw2 = torch.ones([hidden_num2, hidden_num1])
cw3 = torch.ones([hidden_num3, hidden_num2])
cw4 = torch.ones([out_num, hidden_num3])

cw1, cw2, cw3, cw4 = cw1.to(device), cw2.to(device), cw3.to(device), cw4.to(device)

In [ ]:
featdict_grow_and_prune = {
    "GSR": list(np.arange(3900,3960)),
    "Temp": list(np.arange(3960,4020)),
    "IBI" : list(np.arange(4080,4140)),
    "Ox" : list(np.arange(4146,4147)),
    "BP" : list(np.arange(4160,4162)),
    "Q": list(np.arange(4147,4158))
}

indexes_grow_prune = np.array(list(chain.from_iterable(featdict_grow_and_prune.values()))).reshape(-1)
print('number of features:', len(indexes_grow_prune))

In [ ]:
# Loading train, and validation sets
dataset = 'KDE'
batch_size = 256
global_epoch = 0
name = dataset + '_mlp'

In [ ]:
# ==============
generator2 = torch.Generator(device=device)
generator2.manual_seed(RANDOM_STATE)

generator3 = torch.Generator(device='cpu')
generator3.manual_seed(RANDOM_STATE)

np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

torch.manual_seed(RANDOM_STATE)
torch.cuda.manual_seed(RANDOM_STATE)
# ==============

trainloader, validationloader = loadData(path_to_real_data, indexes_grow_prune, mode='train', batch_size=batch_size, generator=generator3)

model = Net3(in_num, out_num, hidden_num1, hidden_num2, hidden_num3)
model = model.to(device)
criterion = nn.CrossEntropyLoss()

flag = 0
best_acc_prune = 0

best_acc_test = 0
best_acc_train = 0

optimizer = torch.optim.SGD(model.parameters(), lr=5e-3, momentum=0.9)
model.load_state_dict(checkpoint['model_state_dict'])

for i in range(1): # number of iterations
    print('====== Prune ======')
    # num in pruneMas shows the final number of connections in model
    # it needs to be decided by trial and error process
    pruneMask([model.fc1.weight.data, model.fc2.weight.data, model.fc3.weight.data, model.fc4.weight.data],
              [cw1, cw2, cw3, cw4], num = 25000)
    flag = 1
    displayConnection([('cw1', cw1), ('cw2', cw2), ('cw3', cw3), ('cw4', cw4)])
    train(20)
    print('====== Grow ======')
    growMask([cw1, cw2, cw3, cw4], mode='partial', ratio=[0.2, 0.2, 0.5, 0.5], generator=generator2)
    flag = 0
    displayConnection([('cw1', cw1), ('cw2', cw2), ('cw3', cw3), ('cw4', cw4)])
    train(20)

In [ ]:
# Loading the model, and evaluating on the validation set
path_to_load = f'{path_dnn3}/KDE_mlp'
load_model(path_to_load, mode='best')
nonzero_connections = displayConnection([('cw1', cw1), ('cw2', cw2), ('cw3', cw3), ('cw4', cw4)])
disp_acc()

### Task 3b: compare the DNN3 size vs orignal DNN2 (1 point)

As a result of grow and prune, you get a compact network with higher accuracy. 

NOTE: your results may be different from CovidDeep paper, since we decreased number of training epochs for faster training. But the DNN3 accuracy should be higher than DNN2.

Calculate the compression ratio for the number of parameters vs the intial network (the number should be > 1)

In [ ]:
# your code is here

### Task 3c: compare the DNN3 accuracy vs orignal DNN2 accuracy (1 point)

Display the result for DNN2.

In this code, use saved_metrics corresponding DNN2 to display the results for DNN2.

In [ ]:
# your code is here